In [75]:
import math
import random
import scipy.stats as sp

n = 10
lmbd = 0.1



y = []

def get_expected_value_theory():
    return 1/lmbd

def get_expected_value_practice():
    #return (1/n)*sum(y)
    return sp.describe(y).mean

def get_err (theory, practice):
    return (abs(theory-practice)/theory)*100

def get_expected_value_err():
    return get_err(get_expected_value_theory(), get_expected_value_practice())

def get_dispersion_theory():
    return 1/(lmbd**2) 

def get_dispersion_practice():
    squared_deviation = []
    for yi in y:
        squared_deviation.append( abs(yi-get_expected_value_practice()) )    
    return sum(squared_deviation)

def get_dispersion_err():
    return get_err(get_dispersion_theory(), get_dispersion_practice())



def get_y (x):
    return (-1)*(math.log(1-x)/lmbd)


for i in range(0,n):
    y.append( get_y(random.randrange(0, 100, 1)/100) )

print(get_expected_value_err())
print(get_dispersion_err())

30.458059111747577
55.03225233364889


In [64]:
! git add * 
! git commit -m !
! git push

The file will have its original line endings in your working directory
The file will have its original line endings in your working directory


[main 6713c1c] !
 2 files changed, 168 insertions(+), 13 deletions(-)


To https://github.com/russianSlave/queuing_theory_lab1.git
   9b61b53..6713c1c  main -> main
